In [ ]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import Word2Vec
from time import time

In [ ]:
# Load the datasets 
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bytes-to-text.csv', names=['raw-bytes', 'tag'], header=None)
print('Data Loaded')

Data Loaded


In [2]:
df['length'] = df['raw-bytes'].str.len()
lengths = df['length']
todel = lengths[lengths>50000]
df = df.drop(todel.index)
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)
df

In [ ]:
df[df['tag']=='BFSSH']

,raw-bytes,tag,length
0,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,590
1,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,5363
2,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,5363
3,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,5363
4,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,590
...,...,...,...
3589,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,5363
3590,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,5363
3591,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,5363
3592,4500 0034 0000 4000 4006 fd7c c0a8 057a 83ca f...,BFSSH,5363


In [ ]:
df[df['tag']=='HTTPDos']

,raw-bytes,tag,length
3594,4500 0028 07cf 0000 8006 abcb c0a8 0375 c0a8 0...,HTTPDos,115
3595,4500 002c 0000 4000 4006 b08e c0a8 057a c0a8 0...,HTTPDos,230
3596,4500 002c 0000 4000 4006 b190 c0a8 057a c0a8 0...,HTTPDos,230
3597,4500 002c 0000 4000 4006 b29c c0a8 057a c0a8 0...,HTTPDos,230
3598,4500 0028 d6ea 0000 8006 dcb1 c0a8 0373 c0a8 0...,HTTPDos,115
...,...,...,...
6697,4500 0028 0538 0000 8006 ae62 c0a8 0375 c0a8 0...,HTTPDos,115
6698,4500 002c 0000 4000 4006 b197 c0a8 057a c0a8 0...,HTTPDos,460
6699,4500 0028 2467 0000 8006 8f36 c0a8 0372 c0a8 0...,HTTPDos,115
6700,4500 0028 2453 0000 8006 8f4a c0a8 0372 c0a8 0...,HTTPDos,115


In [ ]:
df[df['tag']=='Infilt']

,raw-bytes,tag,length
6702,4500 0028 0000 4000 4006 b195 c0a8 057a c0a8 0...,Infilt,230
6703,4500 0028 81da 0000 8006 34d7 c0a8 0165 c0a8 0...,Infilt,115
6704,4500 0028 bbf2 0000 8006 f9b9 c0a8 026a c0a8 0...,Infilt,115
6705,4500 0028 f0c6 0000 8006 c4df c0a8 0270 c0a8 0...,Infilt,115
6706,4500 0028 bbd5 0000 8006 f9d6 c0a8 026a c0a8 0...,Infilt,115
...,...,...,...
16112,4500 0028 4088 0000 8006 710c c0a8 057b c0a8 0...,Infilt,230
16113,4500 0028 f1bf 0000 8006 c3e6 c0a8 0270 c0a8 0...,Infilt,115
16114,4500 0028 4ddf 0000 8006 67c9 c0a8 026e c0a8 0...,Infilt,115
16115,4500 0028 64d6 0000 8006 50d3 c0a8 026d c0a8 0...,Infilt,115


In [ ]:
df[df['tag']=='Normal']

,raw-bytes,tag,length
16117,4500 0030 675d 4000 ef06 8deb 42eb 8f76 c0a8 0...,Normal,6728
16118,4500 0030 e3dd 4000 3706 07c7 4136 51cc c0a8 0...,Normal,13890
16119,45a0 0030 0000 4000 2906 442b 3dd5 0c10 c0a8 0...,Normal,1648
16120,4500 0030 0000 4000 3206 1f58 3ec5 289d c0a8 0...,Normal,1933
16121,4500 0030 0000 4000 3206 c3b8 4314 7ee3 c0a8 0...,Normal,4345
...,...,...,...
26759,4500 0030 1d1c 4000 f206 f6f1 0447 ad59 c0a8 0...,Normal,9346
26760,4500 0030 c7a7 4000 3706 2495 4136 5134 c0a8 0...,Normal,863
26761,4500 0030 09a0 4000 f206 0a6e 0447 ad59 c0a8 0...,Normal,5095
26762,4500 002c 0a82 0000 f006 eb16 3ff5 d120 c0a8 0...,Normal,5133


In [3]:
df['target'] = df['tag'].map({'Normal': 0, 'Infilt': 1, 'BFSSH': 2, 'HTTPDos':3})

data = df['raw-bytes'].to_numpy()
targets = df['target'].to_numpy()

all_text = ' '.join(data)
all_words = all_text.split()

from collections import Counter
counts = Counter(all_words)
vocab = sorted(counts, key=counts.get, reverse=True)
print('vocab length: ', len(vocab))

vocab_to_int = {word:i for i, word in enumerate(vocab[:10000], 1)}

pickle.dump(vocab_to_int, open('/content/drive/My Drive/Colab Notebooks/datasets/vocab_to_int.p', "wb"))

print('Vocab to Int save to drive')

limit_bytes_as_vocab = []
for raw in data:
    limit_bytes_as_vocab.append([word if word in vocab_to_int else 'unkown' for word in raw.split()])
print('Process Complete')

In [ ]:
# Trim the larger rows to 1200 size

'Lets add the 0 padding to the datasets for training the word2vec model'
text_data = []
for i, line in enumerate(limit_bytes_as_vocab):
  text_list = line
  if len(text_list)<1200:
    text_list.extend(['0']*(1200-len(text_list)))
    text_data.append(text_list)
  else:
    text_data.append(text_list[:1200])

In [ ]:
# targets = df['target']
targets = targets.reshape(-1, 1)

test_index=[]
normal=np.random.choice(range(16117, 26763), 1000)
test_index.extend(set(normal))

Inflit=np.random.choice(range(6702, 16116), 900)
test_index.extend(set(Inflit))

http=np.random.choice(range(3594, 6701), 400)
test_index.extend(set(http))

bfssh=np.random.choice(range(0, 3593), 350)
test_index.extend(set(bfssh))

len(test_index)

train_x = []
test_x = []

for i in range(26764):
  if i in test_index:
    test_x.append(text_data[i])
  else:
    train_x.append(text_data[i])


train_y = targets[list(set(range(0, 26763)).symmetric_difference(set(test_index))), :]
test_y = targets[test_index, :]
# print('test_x: ', test_x.shape)
# print('train_x: ', train_x.shape)
print('Len of train data: ', len(train_x))
print('Len of test data: ', len(test_x))
print('train_y: ', train_y.shape)
print('test_y: ', test_y.shape)

Len of train data:  24244
Len of test data:  2520
train_y:  (24243, 1)
test_y:  (2520, 1)


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class lstm_block(nn.Module):

    def __init__(self, embedding_model, in_channels=1000, hidden_dim=512, hidden_dim2=256, num_payers=2, out_channels=4):
        super(lstm_block, self).__init__()

        self.embds = embedding_model

        self.lstm1 = nn.LSTM(in_channels, hidden_dim, num_payers, batch_first=True, dropout=0.4)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim2, num_payers, batch_first=True, dropout=0.4)

        self.fc1 = nn.Linear(hidden_dim2, out_channels)

    def forward(self, x):

        out = self.embds[x]
        out = torch.FloatTensor(out)
        out = out.view(1,300, -1)

        out, hn = self.lstm1(out)
        
        out, hn2 = self.lstm2(out)
        
        out = out[:, -1, :]
        out = out.view(1, -1)
        out = self.fc1(out)
        
        return out

In [ ]:
embedding_model = Word2Vec.load('/content/drive/My Drive/Colab Notebooks/datasets/train-word2vec-model.model')

print('\n\nLOSS OF LAST EPOCH\n')
embedding_model.get_latest_training_loss()
# embedding_model.wv.vocab
# embedding_model.wv.index2entity

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL




LOSS OF LAST EPOCH



0.0

In [ ]:
output_size = 4
embedding_dim = 1200
hidden_dim = 256
n_layers = 2
model = lstm_block(embedding_model, in_channels=1200, hidden_dim=512, hidden_dim2=256, num_payers=2, out_channels=4)

print(model)

lstm_block(
  (lstm1): LSTM(1200, 512, num_layers=2, batch_first=True, dropout=0.4)
  (lstm2): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.4)
  (fc1): Linear(in_features=256, out_features=4, bias=True)
)


In [ ]:
lr=0.01

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
def train_model(model, n_epochs,train_data,target_data,test_data_x, test_data_y, optimizer):
    
    #global variable 
    N_test=len(test_data_x)
    accuracy_list=[]
    loss_list=[]

    for epoch in range(n_epochs):
        total_loss = 0
        count = 0
        for x, y in zip(train_data, target_data):
            model.train()
            count +=1
            optimizer.zero_grad()
            
            z = model(x)
            loss = criterion(z, torch.LongTensor(y))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if count%100==0:
              print('Epoch: ',epoch,'Row Number', count, ' Loss: ', total_loss/count)
        
        print('\n')    
        loss_list.append(total_loss/len(train_data))
        

        correct=0
        #perform a prediction on the test  data  
        for x_test, y_test in zip(test_data_x, test_data_y):
            model.eval()
            z = model(x_test)
            _, yhat = torch.max(z.data, 1)
            correct += (yhat == torch.LongTensor(y_test)).sum().item()
        accuracy = correct / N_test
        accuracy_list.append(accuracy)
        print('Epoch: ', epoch, 'Loss: ', total_loss/len(train_data), 'Accuracy: ', accuracy)
     
    return accuracy_list, loss_list

In [6]:
accuracy_list_normal, loss_list_normal=train_model(model=model,n_epochs=2,train_data=train_x,target_data=train_y, test_data_x= test_x[:10], test_data_y=test_y[:10], optimizer=optimizer)
print('Finished')

# Plot the accuracy and loss
plt.plot(accuracy_list_normal, 'b', label='Accuracy ')
plt.plot(loss_list_normal, 'r', label='loss')
plt.xlabel('Iteration')
plt.title("Loss")
plt.legend()